In [1]:
from ncpol2sdpa import generate_variables, SdpRelaxation
from sympy import *
#import numpy as np

In [2]:
class Variables:

    variables = []
    
    _num_calls = 0
    
    @staticmethod
    def get_new(num):
        """
        num -- number of new variables requested
        """   
        variables = Variables.variables
        
        # generate new variables
        new_variables = generate_variables('u_{}'.format(Variables._num_calls), num)
        
        Variables._num_calls += 1
        
        # save them
        variables.extend(new_variables)
        
        return new_variables
    
    def __init__(self):
        raise Exception("This class is a singleton!")

In [3]:
def poly_approx_single_step_propagator(*, H0, V, dt, Variables, **kwargs):
    """
    H0 -- the field free Hamiltonian
    V -- the interaction operator
    dt -- time step
    Variables -- generator of new time variables
    """
    u = Variables.get_new(1)[0]
    
    # use the linear approximation to the matrix exponent
    X = -I * dt * (H0 + u * V)
    return eye(H0.shape[0]) + X #+ X ** 2 / 2
    
def poly_approx_for_propagator(*, n_tsteps, **kwargs):
    """
    n_tsteps -- number of time steps to reach final time
    """
    assert n_tsteps > 1
    U = poly_approx_single_step_propagator(**kwargs)
    
    for _ in range(n_tsteps - 1):
        U = poly_approx_single_step_propagator(**kwargs) @ U
        
    return U

def get_final_state(*, psi0, **kwargs):
    """
    psi0 -- the initial state
    """
    return poly_approx_for_propagator(**kwargs) @ psi0

# Define quantum systems

In [4]:
sys_params = dict(
    # the field free Hamiltonian
    H0 = Matrix([[0, 1], [1, 0]]), #zeros(2, 2), #Matrix([0, 0]),
    
    # the interaction operator
    V = Matrix([[1, 0], [0, -1]]), #eye(2),
    
    # initial condition
    psi0 = Matrix([[1], [0]]),
    
    # time step
    dt = Rational(1, 1),
    
    # number of time steps
    n_tsteps = 4,
    
    # singleton to yield new optimization variables
    Variables = Variables,
)

# Generate polynomial approximation for the propagator

# good case

In [5]:
psi_fin = get_final_state(**sys_params)

In [18]:
# , 
obj = (psi_fin - Matrix([[-2+50*I], [-6+32*I]])).norm() ** 2


In [19]:
#simplify(psi_fin.subs(zip(Variables.variables, range(3))))
obj

u_0**2*u_1**2*u_2**2*u_3**2 + 2*u_0**2*u_1**2*u_2**2 + 2*u_0**2*u_1**2*u_3**2 + 4*u_0**2*u_1**2 + 2*u_0**2*u_2**2*u_3**2 + 4*u_0**2*u_2**2 + 4*u_0**2*u_3**2 + 8*u_0**2 + 4*u_0*u_1*u_2*u_3 - 88*u_0*u_1*u_2 - 112*u_0*u_1*u_3 - 128*u_0*u_1 - 88*u_0*u_2*u_3 - 8*u_0*u_2 + 128*u_0*u_3 - 224*u_0 + 2*u_1**2*u_2**2*u_3**2 + 4*u_1**2*u_2**2 + 4*u_1**2*u_3**2 + 8*u_1**2 - 112*u_1*u_2*u_3 - 128*u_1*u_2 - 8*u_1*u_3 + 176*u_1 + 4*u_2**2*u_3**2 + 8*u_2**2 - 128*u_2*u_3 + 224*u_2 + 8*u_3**2 - 176*u_3 + 3564

In [22]:
sdp = SdpRelaxation(Variables.variables)
sdp.get_relaxation(5, objective=obj)
sdp.solve(solver='mosek')

In [23]:
# extract the values of control
if sdp.status == 'optimal':
    print([sdp[_] for _ in sdp.variables])

[1.0299920247164267, 2.007231264171626, 2.981472508644547, 3.9759304648422775]


In [73]:
sdp.primal, sdp.dual

(8.99776987353107e-05, 8.646784965549159e-05)

In [25]:
simplify(
    obj.subs(
        zip(sdp.variables, (sdp[_] for _ in sdp.variables))
    )
)

0.524885136870580

In [16]:
((1 - I) ** 2).evalf()

-2.0*I

In [9]:
psi_fin.subs(
        zip(Variables.variables, [1,2,3])
    )

Matrix([
[               -I*(-I*(1 - I) - I*(1 + 2*I)) + (-1 + (1 - 2*I)*(1 - I))*(1 - 3*I)],
[(I*u_2 + 1)*(-I*(-I*u_0 + 1) - I*(I*u_1 + 1)) - I*((-I*u_0 + 1)*(-I*u_1 + 1) - 1)]])

In [10]:
psi_fin

Matrix([
[(-I*u_2 + 1)*((-I*u_0 + 1)*(-I*u_1 + 1) - 1) - I*(-I*(-I*u_0 + 1) - I*(I*u_1 + 1))],
[ (I*u_2 + 1)*(-I*(-I*u_0 + 1) - I*(I*u_1 + 1)) - I*((-I*u_0 + 1)*(-I*u_1 + 1) - 1)]])

In [26]:
simplify(psi_fin[0].subs(
        zip(Variables.variables, [1,2,1,2])
    ))

-10 + 12*I

In [16]:
list(zip(Variables.variables, [1,2,3]))

[(u_0, 1), (u_1, 2), (u_2, 3)]

In [64]:
[_.subs(zip(Variables.variables, [1,2,3,1])).simplify() for _ in psi_fin]

[-8 + 11*I, 3 + 20*I]

In [23]:
Matrix([[-13 + 2*I], [4 + 3*I]])

Matrix([
[-13 + 2*I],
[  4 + 3*I]])

In [7]:
simplify(psi_fin)

Matrix([
[   -(I*u_0 - 1)*(I*u_1 - 1) - (I*u_2 + 1)*(-I*u_0 + I*u_1 + 2) + (I*u_3 - 1)*(-I*u_0 + I*u_1 + (I*u_2 - 1)*((I*u_0 - 1)*(I*u_1 - 1) - 1) + 2) + 1],
[I*(-I*u_0 + I*u_1 + (I*u_2 - 1)*((I*u_0 - 1)*(I*u_1 - 1) - 1) - (I*u_3 + 1)*((I*u_0 - 1)*(I*u_1 - 1) + (I*u_2 + 1)*(-I*u_0 + I*u_1 + 2) - 1) + 2)]])

# not good

In [27]:
# , 
obj = (psi_fin - Matrix([[-10+12*I], [-4+8*I]])).norm() ** 2

In [42]:
sdp = SdpRelaxation(Variables.variables)
sdp.get_relaxation(8, objective=obj)
sdp.solve(solver='mosek')

In [43]:
# extract the values of control
if sdp.status == 'optimal':
    print([sdp[_] for _ in sdp.variables])

[1.826129233606543, -1.2148344330854794, -0.8163887544402191, -1.8153311470626372]


In [44]:
sdp.primal, sdp.dual

(0.00031512017568502415, 0.00031319318168243626)

In [62]:
vals = list(zip(sdp.variables, (sdp[_] for _ in sdp.variables)))

In [64]:
# value of the objective function
simplify(obj.subs(vals))

3.79601601561677

In [67]:
# the final state
Matrix([
    [simplify(c.subs(vals))] for c in psi_fin
])

Matrix([
[-8.7167611819417 + 10.8283161772528*I],
[-4.19537122394937 + 7.1407554156062*I]])